In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import openpyxl as pxl
path="C:/Users/diersh/OneDrive - EWI an der Universität zu Köln/Dokumente/Promotion/ATIS3/data/"

## Prep time series data

In [2]:
df1=pd.read_csv(path+"Data_15min_TransnetBW.csv")
df1["Begin"]=df1["Begin"].map(lambda x: pd.to_datetime(x.strip(), format='%d.%m.%Y %H:%M'))
df1.set_index("Begin", inplace=True)
df1=df1.resample("H").mean()
df1.drop(columns=["Unnamed: 0"], inplace=True)
df3=pd.read_csv(path+"data_60min.csv")
df3["Begin"]=df3["Begin"].map(lambda x: pd.to_datetime(str(x)[:-9].strip(), format='%Y-%m-%d %H:%M'))
df3.set_index("Begin", inplace=True)
df3=df3[['DA_Price', 'auc_15min_hour_price', 'ID3_price', 'FC_DA_Load', 
       'Total_load','Prognose_Wind', 'Ist_Wind', 'Prognose_PV', 'Ist_PV']]  #only use relavant variables
df1["merge"]=range(len(df1)) # support variable
df3["merge"]=range(len(df3)) # support variable
ix=df3.index  # save index for later
df3=df3.merge(df1, on="merge")
df3.index=ix
df3.drop(columns=["merge"], inplace=True)
# simplyfy labels
df3.rename(columns={'DA_Price': "p_da", 'auc_15min_hour_price': "p_15", 'ID3_price':"p_id3", 'FC_DA_Load':"fc_load_de",
'Total_load':"ist_load_de", 'Prognose_Wind':"fc_wind_de", 'Ist_Wind':"ist_wind_de", 'Prognose_PV':"fc_pv_de", 
 'Ist_PV':"ist_pv_de",'Ist_Load_BW':"ist_load_bw", 'Prognose_Load_BW':"fc_load_bw", 'Ist_Wind_BW':"ist_wind_bw",
'Prognose_Wind_BW':"fc_wind_bw", 'Ist_PV_BW':"ist_pv_bw", 'Prognose_PV_BW':"fc_pv_bw"},inplace=True)
df3["p_15"]=df3["p_15"].round(2)  #
df3.loc[df3["fc_wind_bw"]>1600, "fc_wind_bw"]=1600  # cut unplausible values
df3["error_wind_bw"]=(df3["ist_wind_bw"]-df3["fc_wind_bw"]).round(2)  #absolute errors
df3["error_wind_de"]=(df3["ist_wind_de"]-df3["fc_wind_de"]).round(2)
df3["rel_error_wind_bw"]=((df3["ist_wind_bw"]-df3["fc_wind_bw"])/df3["fc_wind_bw"]).round(3)
df3["rel_error_wind_de"]=((df3["ist_wind_de"]-df3["fc_wind_de"])/df3["fc_wind_bw"]).round(3)
df3.replace([np.inf, -np.inf], np.nan, inplace=True)
df3.loc[df3["error_wind_bw"]<-900,"error_wind_bw"]=-900 #cut extrem values
df3.loc[df3["rel_error_wind_bw"]>4,"rel_error_wind_bw"]=4  #cut extrem values
df3["int_ix"]=range(len(df3))
df3["day"]=df3.index.day
df3["hour"]=df3.index.hour+1

C:\Users\diersh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
with pd.ExcelWriter('data_ATIS3.xlsx') as writer:  
    df3.to_excel(writer, sheet_name='time_series')   #save time series data

## Generate Scenarios

In [3]:
# create fc diviation scenarios
excel_book = pxl.load_workbook('data_ATIS3.xlsx')

with pd.ExcelWriter('data_ATIS3.xlsx', engine='openpyxl') as writer:
    
    writer.book = excel_book
# we conduct the analysis for each hour since the fc errors are nout uniformly distributed accross all 24h
        df4=df3[df3["hour"]==h]    
    for h in range(1,25):    
        scen=pd.DataFrame(columns=["neg_u250", "pos_u250", "neg", "pos", "neg_o1250", "pos_o1250"])

        d1=df4.loc[(df4["fc_wind_bw"]>=0)&(df4["fc_wind_bw"]<250),"rel_error_wind_bw"]
        d2=df4.loc[(df4["fc_wind_bw"]>=250)&(df4["fc_wind_bw"]<1250),"rel_error_wind_bw"]
        d3=df4.loc[(df4["fc_wind_bw"]>=1250)&(df4["fc_wind_bw"]<1600),"rel_error_wind_bw"]

        for q in range(1,101,2):
            scen.loc[q, "neg_u250"]=d1[d1<0].quantile(q/100, interpolation="linear")
            scen.loc[q, "pos_u250"]=d1[d1>=0].quantile(q/100, interpolation="linear")
            scen.loc[q, "neg"]=d2[d2<0].quantile(q/100, interpolation="linear")
            scen.loc[q, "pos"]=d2[d2>=0].quantile(q/100, interpolation="linear")
            scen.loc[q, "neg_o1250"]=d3[d3<0].quantile(q/100, interpolation="linear")
            scen.loc[q, "pos_o1250"]=d3[d3>=0].quantile(q/100, interpolation="linear")

        scen.index=range(1,51) 

        # Write the new data to the file without overwriting what already exists
        scen.to_excel(writer, sheet_name='s_q_'+str(h))

        # Save the file
        writer.save()

In [4]:
# create price diviation scenarios

df3["fp_1"]=((df3.p_da-df3.p_15)/df3.p_da).round(3)   #we use realtive diviations
df3["fp_2"]=((df3.p_da-df3.p_id3)/df3.p_da).round(3)
df3.replace([np.inf, -np.inf], np.nan, inplace=True)

t=5   #cut outlines in the sample down to a threshold value
act=t
df3.loc[abs(df3["fp_1"])>t,"fp_1"]=act
df3.loc[abs(df3["fp_1"])<-t,"fp_1"]=-act
df3.loc[abs(df3["fp_2"])>t,"fp_2"]=act
df3.loc[abs(df3["fp_2"])<-t,"fp_2"]=-act

scen2=pd.DataFrame(columns=["delta_15_da", "delta_id3_da"])
d1=df3["fp_1"]
d2=df3["fp_2"]
for q in range(1,101,5):
    scen2.loc[q, "delta_15_da"]=d1.quantile(q/100, interpolation="linear")
    scen2.loc[q, "delta_id3_da"]=d2.quantile(q/100, interpolation="linear")
    
scen2.index=range(1,21) 

excel_book = pxl.load_workbook('data_ATIS3.xlsx')

with pd.ExcelWriter('data_ATIS3.xlsx', engine='openpyxl') as writer:
    writer.book = excel_book
    scen2.to_excel(writer, sheet_name='s_prices')
 # Save the file
    writer.save()